In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/resume-dataset/Resumes.json
/kaggle/input/vocab-bert/vocab/vocab.txt


In [2]:
import os
vocab_path ='/kaggle/input/vocab-bert/vocab/vocab.txt'
if not os.path.isfile(vocab_path):
    print(f"Vocabulary file not found at {vocab_path}")

In [3]:
!pip install seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16161 sha256=ed5e763fba8d0ee3fedc9236cf68b0a3a8bc084e4b9e8292028fb70397e19cc4
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval


In [4]:
pip show seqeval

Name: seqeval
Version: 1.2.2
Summary: Testing framework for sequence labeling
Home-page: https://github.com/chakki-works/seqeval
Author: Hironsan
Author-email: hiroki.nakayama.py@gmail.com
License: MIT
Location: /opt/conda/lib/python3.10/site-packages
Requires: numpy, scikit-learn
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [5]:
import warnings

warnings.filterwarnings("ignore", category=UserWarning, module='seqeval')

In [6]:
import re
import json
import logging
import numpy as np
from tqdm import trange
import torch
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from seqeval.metrics import classification_report
from sklearn.metrics import confusion_matrix

In [7]:
def convert_dataset(filePath):
    """Convert the dataset in json format to the required text and entity format"""
    try:
        training_data = []
        lines = []
        # Open the file using UTF-8 encoding
        with open(filePath, 'r', encoding='utf-8') as f:
            data = json.load(f)  # load the JSON content
            print(type(data))
            for item in data:
                lines.append(json.dumps(item))
            

        for line in lines:
            data = json.loads(line)
            text = data['content'].replace("\n", " ")
            entities = []
            data_annotations = data['annotation']
            if data_annotations is not None:
                for annotation in data_annotations:
                    point = annotation['points'][0]
                    labels = annotation['label']
                    if not isinstance(labels, list):
                        labels = [labels]

                    for label in labels:
                        point_start = point['start']
                        point_end = point['end']
                        point_text = point['text']

                        lstrip_diff = len(point_text) - len(point_text.lstrip())
                        rstrip_diff = len(point_text) - len(point_text.rstrip())
                        if lstrip_diff != 0:
                            point_start = point_start + lstrip_diff
                        if rstrip_diff != 0:
                            point_end = point_end - rstrip_diff
                        entities.append((point_start, point_end + 1, label))
            training_data.append((text, {"entities": entities}))
        return training_data
    except Exception as e:
        logging.exception("Unable to process " +
                          filePath + "\n" + "error = " + str(e))
        return None

In [8]:
def trim_entity_spans(data: list) -> list:
    """Removes leading and trailing white spaces from entity spans.
        The data in spaCy json format is cleaned
    """
    invalid_span_tokens = re.compile(r'\s')

    cleaned_data = []
    for text, annotations in data:
        entities = annotations['entities']
        valid_entities = []
        for start, end, label in entities:
            valid_start = start
            valid_end = end
            while valid_start < len(text) and invalid_span_tokens.match(
                    text[valid_start]):
                valid_start += 1
            while valid_end > 1 and invalid_span_tokens.match(
                    text[valid_end - 1]):
                valid_end -= 1
            valid_entities.append([valid_start, valid_end, label])
        cleaned_data.append([text, {'entities': valid_entities}])
    return cleaned_data

In [9]:
def get_label(offset, labels):
    """The function compares the offset of the token 
    to the character offsets of the labeled spans 
    in labels to determine if the token falls within any labeled span."""
    if offset[0] == 0 and offset[1] == 0:
        return 'O'
    for label in labels:
        if offset[1] >= label[0] and offset[0] <= label[1]:
            return label[2]
    return 'O'

In [10]:
tags_vals = ["UNKNOWN", "O", "Name", "Degree", "Skills", "College Name", "Email Address",
             "Designation", "Companies worked at", "Graduation Year", "Years of Experience", "Location"]

tag2idx = {t: i for i, t in enumerate(tags_vals)}
idx2tag = {i: t for i, t in enumerate(tags_vals)}

In [11]:
print(tags_vals)
print(tag2idx)
print(idx2tag)


['UNKNOWN', 'O', 'Name', 'Degree', 'Skills', 'College Name', 'Email Address', 'Designation', 'Companies worked at', 'Graduation Year', 'Years of Experience', 'Location']
{'UNKNOWN': 0, 'O': 1, 'Name': 2, 'Degree': 3, 'Skills': 4, 'College Name': 5, 'Email Address': 6, 'Designation': 7, 'Companies worked at': 8, 'Graduation Year': 9, 'Years of Experience': 10, 'Location': 11}
{0: 'UNKNOWN', 1: 'O', 2: 'Name', 3: 'Degree', 4: 'Skills', 5: 'College Name', 6: 'Email Address', 7: 'Designation', 8: 'Companies worked at', 9: 'Graduation Year', 10: 'Years of Experience', 11: 'Location'}


In [12]:
def process_resume(data, tokenizer, tag2idx, max_len, is_test=False):
    #data[0] is the text of the resume
    tok = tokenizer.encode_plus(
        data[0],
        max_length=max_len,
        padding ='max_length',
        truncation=True,
        return_tensors="pt",
        return_offsets_mapping=True)
    
    curr_sent = {'orig_labels': [], 'labels': []}

    padding_length = max_len - tok['input_ids'].shape[1]

    if not is_test:
        labels = data[1]['entities']
        labels.reverse()
        for off in tok['offset_mapping'][0]:
            label = get_label(off, labels)
            curr_sent['orig_labels'].append(label)
            curr_sent['labels'].append(tag2idx[label])

        curr_sent['labels'] = curr_sent['labels'] + ([0] * padding_length)

    curr_sent['input_ids'] = tok['input_ids'].squeeze(0).tolist()  + ([0] * padding_length)
    curr_sent['token_type_ids'] = tok['token_type_ids'].squeeze(0).tolist()  + ([0] * padding_length)
    curr_sent['attention_mask'] = tok['attention_mask'].squeeze(0).tolist()  + ([0] * padding_length)
    
    if 'token_type_ids' in tok:
        curr_sent['token_type_ids'] = tok['token_type_ids'].squeeze(0).tolist() + ([0] * padding_length)
    else:
        curr_sent['token_type_ids'] = [0] * max_len
        
    
#     print(curr_sent)

    return curr_sent

In [13]:
class ResumeDataset(Dataset):
    def __init__(self, resume, tokenizer, tag2idx, max_len, is_test=False):
        self.resume = resume
        self.tokenizer = tokenizer
        self.is_test = is_test
        self.tag2idx = tag2idx
        self.max_len = max_len

    def __len__(self):
        return len(self.resume)

    def __getitem__(self, idx):
        data = process_resume(
            self.resume[idx], self.tokenizer, self.tag2idx, self.max_len, self.is_test)
        return {
            'input_ids': torch.tensor(data['input_ids'], dtype=torch.long),
            'token_type_ids': torch.tensor(data['token_type_ids'], dtype=torch.long),
            'attention_mask': torch.tensor(data['attention_mask'], dtype=torch.long),
            'labels': torch.tensor(data['labels'], dtype=torch.long),
            'orig_label': data['orig_labels']
        }

In [14]:
def get_hyperparameters(model, ff):

    # ff: full_finetuning
    if ff:
        param_optimizer = list(model.named_parameters())
        no_decay = ["bias", "gamma", "beta"]
        optimizer_grouped_parameters = [
            {
                "params": [
                    p for n, p in param_optimizer if not any(nd in n for nd in no_decay)
                ],
                "weight_decay_rate": 0.01,
            },
            {
                "params": [
                    p for n, p in param_optimizer if any(nd in n for nd in no_decay)
                ],
                "weight_decay_rate": 0.0,
            },
        ]
    else:
        param_optimizer = list(model.classifier.named_parameters())
        optimizer_grouped_parameters = [
            {"params": [p for n, p in param_optimizer]}]

    return optimizer_grouped_parameters

In [15]:
def get_special_tokens(tokenizer, tag2idx):
    vocab = tokenizer.get_vocab()
    pad_tok = vocab["[PAD]"]
    sep_tok = vocab["[SEP]"]
    cls_tok = vocab["[CLS]"]
    o_lab = tag2idx["O"]

    return pad_tok, sep_tok, cls_tok, o_lab


def annot_confusion_matrix(valid_tags, pred_tags):
    """
    Create an annotated confusion matrix by adding label
    annotations and formatting to sklearn's `confusion_matrix`.
    """

    flat_valid_tags = [item for sublist in valid_tags for item in sublist]
    flat_pred_tags = [item for sublist in pred_tags for item in sublist]

    # Get the unique labels
    header = sorted(list(set(flat_valid_tags + flat_pred_tags)))

    # Compute the confusion matrix
    matrix = confusion_matrix(flat_valid_tags, flat_pred_tags, labels=header)

    # Format the matrix for printing
    mat_formatted = [header[i] + "\t\t\t" +
                     str(row) for i, row in enumerate(matrix)]
    content = "\t" + " ".join(header) + "\n" + "\n".join(mat_formatted)

    return content


def flat_accuracy(valid_tags, pred_tags):
    return (np.array(valid_tags) == np.array(pred_tags)).mean()

In [16]:
def train_and_val_model(
    model,
    tokenizer,
    optimizer,
    epochs,
    idx2tag,
    tag2idx,
    max_grad_norm,
    device,
    train_dataloader,
    valid_dataloader
):

    pad_tok, sep_tok, cls_tok, o_lab = get_special_tokens(tokenizer, tag2idx)

    epoch = 0
    for _ in trange(epochs, desc="Epoch"):
        epoch += 1

        # Training loop
        print("Starting training loop.")
        model.train()
        tr_loss, tr_accuracy = 0, 0
        nb_tr_examples, nb_tr_steps = 0, 0
        tr_preds, tr_labels = [], []

        for step, batch in enumerate(train_dataloader):
            # Add batch to gpu

            # batch = tuple(t.to(device) for t in batch)
            b_input_ids, b_input_mask, b_labels = batch['input_ids'], batch['attention_mask'], batch['labels']
            b_input_ids, b_input_mask, b_labels = b_input_ids.to(
                device), b_input_mask.to(device), b_labels.to(device)

            # Forward pass
            outputs = model(
                b_input_ids,
                token_type_ids=None,
                attention_mask=b_input_mask,
                labels=b_labels,
            )
            loss, tr_logits = outputs[:2]

            # Backward pass
            loss.backward()

            # Compute train loss
            tr_loss += loss.item()
            nb_tr_examples += b_input_ids.size(0)
            nb_tr_steps += 1

            # Subset out unwanted predictions on CLS/PAD/SEP tokens
            preds_mask = (
                (b_input_ids != cls_tok)
                & (b_input_ids != pad_tok)
                & (b_input_ids != sep_tok)
            )

            tr_logits = tr_logits.cpu().detach().numpy()
            tr_label_ids = torch.masked_select(b_labels, (preds_mask == 1))
            preds_mask = preds_mask.cpu().detach().numpy()
            tr_batch_preds = np.argmax(tr_logits[preds_mask.squeeze()], axis=1)
            tr_batch_labels = tr_label_ids.to("cpu").numpy()
            tr_preds.extend(tr_batch_preds)
            tr_labels.extend(tr_batch_labels)

            # Compute training accuracy
            tmp_tr_accuracy = flat_accuracy(tr_batch_labels, tr_batch_preds)
            tr_accuracy += tmp_tr_accuracy

            # Gradient clipping
            torch.nn.utils.clip_grad_norm_(
                parameters=model.parameters(), max_norm=max_grad_norm
            )

            # Update parameters
            optimizer.step()
            model.zero_grad()

        tr_loss = tr_loss / nb_tr_steps
        tr_accuracy = tr_accuracy / nb_tr_steps

        # Print training loss and accuracy per epoch
        print(f"Train loss: {tr_loss}")
        print(f"Train accuracy: {tr_accuracy}")

        """
        Validation loop
        """
        print("Starting validation loop.")

        pred_tags = []
        valid_tags = []
        eval_loss, eval_accuracy = 0, 0  
        nb_eval_steps, nb_eval_examples = 0, 0
        predictions, true_labels = [], []
        
        for i, batch in enumerate(valid_dataloader):
            b_input_ids, b_input_mask, b_labels = batch['input_ids'], batch['attention_mask'], batch['labels']
            b_input_ids, b_input_mask, b_labels = b_input_ids.to(device), b_input_mask.to(device), b_labels.to(device)
        
            with torch.no_grad():
                outputs = model(
                    b_input_ids,
                    token_type_ids=None,
                    attention_mask=b_input_mask,
                    labels=b_labels,
                )
                tmp_eval_loss, logits = outputs[:2]
            
                preds_mask = (
                    (b_input_ids != cls_tok)
                    & (b_input_ids != pad_tok)
                    & (b_input_ids != sep_tok)
                )
                preds_mask = preds_mask.cpu().detach()
            
                # Keep logits as a tensor for any operation requiring it
                logits = logits.cpu().detach()
            
                # Use preds_mask with torch.masked_select
                label_ids = torch.masked_select(b_labels.cpu(), preds_mask)
                val_batch_preds = np.argmax(logits[preds_mask].numpy(), axis=1)  # Convert logits to numpy here, after masking
                val_batch_labels = label_ids.to("cpu").numpy()
                predictions.extend(val_batch_preds)
                true_labels.extend(val_batch_labels)
            
                eval_loss += tmp_eval_loss.item()
            
                tmp_eval_accuracy = flat_accuracy(
                    val_batch_labels, val_batch_preds)
                eval_accuracy += tmp_eval_accuracy
            
                nb_eval_examples += b_input_ids.size(0)
                nb_eval_steps += 1
            
                    
            # Convert logits and labels to numpy
            logits = logits.detach().cpu().numpy()
            label_ids = b_labels.to('cpu').numpy()
            input_ids = b_input_ids.cpu().numpy()
        
            for i in range(len(label_ids)):
                # Apply mask to remove special tokens
                seq_pred = []
                seq_true = []
                for j in range(len(label_ids[i])):
                    if input_ids[i][j] not in [cls_tok, pad_tok, sep_tok]:  
                        seq_pred.append(idx2tag[np.argmax(logits[i][j])])  
                        seq_true.append(idx2tag[label_ids[i][j]])  
                
                # Append sequence-level predictions and true labels
                pred_tags.append(seq_pred)
                valid_tags.append(seq_true)
        
        # Now pred_tags and valid_tags are lists of lists (one sublist per sequence)
        cl_report = classification_report(valid_tags, pred_tags)
        conf_mat = annot_confusion_matrix(valid_tags, pred_tags)
        
        # Report metrics
        eval_loss = eval_loss / nb_eval_steps
        eval_accuracy = flat_accuracy([item for sublist in valid_tags for item in sublist],
                                      [item for sublist in pred_tags for item in sublist])
        
        print(f"Validation loss: {eval_loss}")
        print(f"Validation Accuracy: {eval_accuracy}")
        print(f"Classification Report:\n {cl_report}")
        print(f"Confusion Matrix:\n {conf_mat}")


In [17]:
import numpy as np
import torch
from transformers import BertForTokenClassification, BertTokenizerFast
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from torch.optim import Adam

output_path = '/kaggle/working'

MAX_LEN = 500
EPOCHS = 5
MAX_GRAD_NORM = 1.0
MODEL_NAME = 'bert-base-uncased'
TOKENIZER = BertTokenizerFast.from_pretrained(MODEL_NAME, do_lower_case=True)
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# data = trim_entity_spans(convert_goldparse('data/Resumes.json'))
data = convert_dataset('/kaggle/input/resume-dataset/Resumes.json')
print(data[0])

# print(len(data))
# print(data)
if data is not None:
    data = trim_entity_spans(data)
    print(data[0])
else:
    print("No data to process due to earlier errors.")

total = len(data)
# print(f"Total resumes: {total}")
train_data, val_data = data[:180],data[180:]

print("Creating the datasets...")
train_d = ResumeDataset(train_data, TOKENIZER, tag2idx, MAX_LEN)
print(train_d[0])
val_d = ResumeDataset(val_data, TOKENIZER, tag2idx, MAX_LEN)

train_sampler = RandomSampler(train_d)
train_dl = DataLoader(train_d, sampler=train_sampler, batch_size=8)

val_dl = DataLoader(val_d, batch_size=4)

model = BertForTokenClassification.from_pretrained(
    MODEL_NAME, num_labels=len(tag2idx))
model.to(DEVICE)
optimizer_grouped_parameters = get_hyperparameters(model, True)
optimizer = Adam(optimizer_grouped_parameters, lr=3e-5)



tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


<class 'list'>
("Abhishek Jha Application Development Associate - Accenture  Bengaluru, Karnataka - Email me on Indeed: indeed.com/r/Abhishek-Jha/10e7a8cb732bc43a  • To work for an organization which provides me the opportunity to improve my skills and knowledge for my individual and company's growth in best possible ways.  Willing to relocate to: Bangalore, Karnataka  WORK EXPERIENCE  Application Development Associate  Accenture -  November 2017 to Present  Role: Currently working on Chat-bot. Developing Backend Oracle PeopleSoft Queries for the Bot which will be triggered based on given input. Also, Training the bot for different possible utterances (Both positive and negative), which will be given as input by the user.  EDUCATION  B.E in Information science and engineering  B.v.b college of engineering and technology -  Hubli, Karnataka  August 2013 to June 2017  12th in Mathematics  Woodbine modern school  April 2011 to March 2013  10th  Kendriya Vidyalaya  April 2001 to March 2011

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [18]:
print("Training and Validating...")
train_and_val_model(
    model,
    TOKENIZER,
    optimizer,
    EPOCHS,
    idx2tag,
    tag2idx,
    MAX_GRAD_NORM,
    DEVICE,
    train_dl,
    val_dl
)

torch.save(
    {
        "model_state_dict": model.state_dict()
    },
    f'{output_path}/model-state.bin',
)


Training and Validating...


Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

Starting training loop.
Train loss: 0.8714336286420408
Train accuracy: 0.7829989026064017
Starting validation loop.


Epoch:  20%|██        | 1/5 [08:25<33:43, 505.90s/it]

Validation loss: 0.6161925315856933
Validation Accuracy: 0.819594383775351
Classification Report:
                     precision    recall  f1-score   support

               ame       0.00      0.00      0.00        41
ears of Experience       0.00      0.00      0.00         5
             egree       0.00      0.00      0.00        35
        esignation       0.00      0.00      0.00        89
             kills       0.18      0.02      0.03       869
      mail Address       0.56      0.53      0.55      1130
           ocation       0.00      0.00      0.00        54
       ollege Name       0.00      0.00      0.00        33
ompanies worked at       0.00      0.00      0.00        59
    raduation Year       0.00      0.00      0.00        16

         micro avg       0.53      0.26      0.35      2331
         macro avg       0.07      0.05      0.06      2331
      weighted avg       0.34      0.26      0.28      2331

Confusion Matrix:
 	College Name Companies worked at Degre

...

Validation loss: 0.3593557670712471
Validation Accuracy: 0.8414976599063962
Classification Report:
                     precision    recall  f1-score   support

               ame       0.25      0.46      0.32        41
ears of Experience       0.00      0.00      0.00         5
             egree       0.10      0.20      0.14        35
        esignation       0.17      0.21      0.19        89
             kills       0.39      0.72      0.50       869
      mail Address       0.85      0.75      0.80      1130
           ocation       0.18      0.28      0.22        54
       ollege Name       0.04      0.15      0.06        33
ompanies worked at       0.07      0.12      0.09        59
    raduation Year       0.00      0.00      0.00        16

         micro avg       0.48      0.67      0.56      2331
         macro avg       0.21      0.29      0.23      2331
      weighted avg       0.58      0.67      0.60      2331

Confusion Matrix:
 	College Name Companies worked at Degr

Epoch:  80%|████████  | 4/5 [34:00<08:29, 509.53s/it]

Validation loss: 0.2779960721731186
Validation Accuracy: 0.8925429017160686
Classification Report:
                     precision    recall  f1-score   support

               ame       0.83      0.85      0.84        41
ears of Experience       0.00      0.00      0.00         5
             egree       0.13      0.11      0.12        35
        esignation       0.20      0.15      0.17        92
             kills       0.84      0.48      0.61       869
      mail Address       0.83      0.76      0.79      1130
           ocation       0.40      0.40      0.40        53
       ollege Name       0.06      0.12      0.08        33
ompanies worked at       0.10      0.13      0.12        60
    raduation Year       0.00      0.00      0.00        16

         micro avg       0.73      0.59      0.65      2334
         macro avg       0.34      0.30      0.31      2334
      weighted avg       0.75      0.59      0.65      2334

Confusion Matrix:
 	College Name Companies worked at Degr

Epoch: 100%|██████████| 5/5 [42:19<00:00, 507.85s/it]

Validation loss: 0.30792873799800874
Validation Accuracy: 0.8828705148205929
Classification Report:
                     precision    recall  f1-score   support

               ame       0.86      0.90      0.88        41
ears of Experience       0.00      0.00      0.00         5
             egree       0.41      0.34      0.38        35
        esignation       0.21      0.22      0.22        89
             kills       0.80      0.31      0.45       869
      mail Address       0.87      0.62      0.73      1130
           ocation       0.53      0.50      0.51        54
       ollege Name       0.22      0.39      0.28        33
ompanies worked at       0.14      0.31      0.19        59
    raduation Year       0.00      0.00      0.00        16

         micro avg       0.71      0.47      0.57      2331
         macro avg       0.40      0.36      0.36      2331
      weighted avg       0.77      0.47      0.57      2331

Confusion Matrix:
 	College Name Companies worked at Deg

In [19]:
checkpoint = torch.load(f'{output_path}/model-state.bin')
model.load_state_dict(checkpoint['model_state_dict'])

/tmp/ipykernel_30/2027259983.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(f'{output_path}/model-state.bin')


<All keys matched successfully>

In [20]:
import shutil

shutil.make_archive('/kaggle/working/model', 'zip', '/kaggle/working', 'model-state.bin')

'/kaggle/working/model.zip'

In [21]:
torch.save(
    {
        "model_state_dict": model.state_dict()
    },
    f'{output_path}/model-state.pth', 
)